In [1]:
import tabula
import pandas as pd
import pickle

## Obtain PDF

file updates daily: http://stopcoronavirus.mcgm.gov.in/assets/docs/Dashboard.pdf

## Scrape PDF

Obtain tables from file. Convert and save tables as csv files

* using tablua for scraping: http://theautomatic.net/2019/05/24/3-ways-to-scrape-tables-from-pdfs-with-python/

* https://nbviewer.jupyter.org/github/chezou/tabula-py/blob/master/examples/tabula_example.ipynb

In [2]:
file = "http://stopcoronavirus.mcgm.gov.in/assets/docs/Dashboard.pdf"

### Ward-wise new cases

Positive cases for the last 7 days 

In [3]:
new_case_growth = tabula.read_pdf(file, pages=24, multiple_tables=False)

new_case_growth[0]

,Date of report,RC,RN,RS,HW,A,PS,D,C,MW,...,GN,HE,S,L,GS,KE,N,B,ME,Grand Total
0,Area>>>,Borivali,Dahisar,Kandivali,Bandra,Colaba,Goregaon,Grant\rRoad,Marine\rLines,Chembur\rW,...,Dadar,Khar,Bhandup,Kurla,Elphinstone,Andheri\rE,Ghatkopar,Sandhurst\rRoad,Chembur\rE,NaN
1,As on 16th Aug,6675,3272,6043,3519,3133,4248,5348,1798,3812,...,7582,4439,7066,5597,5620,8042,6657,1150,4520,125079.0
2,17-Aug,57,10,48,40,7,17,34,12,21,...,50,19,30,29,34,28,39,6,17,703.0
3,18-Aug,83,36,68,30,28,37,56,7,25,...,28,18,42,28,33,36,37,4,11,860.0
4,19-Aug,79,53,58,38,30,34,63,18,31,...,76,31,41,32,33,52,49,5,31,1050.0
5,20-Aug,161,48,84,39,27,43,65,13,42,...,71,35,38,45,37,65,39,3,22,1200.0
6,21-Aug,147,54,102,54,45,77,56,27,40,...,47,45,57,35,24,73,33,10,32,1324.0
7,22-Aug,129,51,70,32,52,51,49,17,46,...,41,20,57,37,42,45,31,12,24,1085.0
8,23-Aug,77,24,59,50,43,50,45,25,24,...,31,29,40,28,30,29,37,5,20,919.0
9,As on 23rd Aug,7408,3548,6532,3802,3365,4557,5716,1917,4041,...,7926,4636,7371,5831,5853,8370,6922,1195,4677,132220.0


In [4]:
new_cases = new_case_growth[0]
new_cases.drop(labels=11, inplace=True)

### Save ward identifier with corresponding ward name in a dictionary

In [7]:
wards = new_cases.iloc[0][1:] # series of ward names, index is identifiers
identifiers = new_cases.iloc[0][1:].index
ward_identifiers = {}

In [23]:
wards.drop('Grand Total', inplace=True)

wards

RC           Borivali
RN            Dahisar
RS          Kandivali
HW             Bandra
A              Colaba
PS           Goregaon
D         Grant\rRoad
C       Marine\rLines
MW         Chembur\rW
E             Byculla
T              Mulund
KW         Andheri\rW
FN            Matunga
PN              Malad
FS              Parel
GN              Dadar
HE               Khar
S             Bhandup
L               Kurla
GS        Elphinstone
KE         Andheri\rE
N           Ghatkopar
B     Sandhurst\rRoad
ME         Chembur\rE
Name: 0, dtype: object

In [24]:
# for loop needs to be fixed

for ward in wards:
    text = str(ward)
    name = text.replace('\r',' ')
    index = wards[wards==text].index[0]
    ward_identifiers[index] = name

In [26]:
ward_identifiers

{'RC': 'Borivali',
 'RN': 'Dahisar',
 'RS': 'Kandivali',
 'HW': 'Bandra',
 'A': 'Colaba',
 'PS': 'Goregaon',
 'D': 'Grant Road',
 'C': 'Marine Lines',
 'MW': 'Chembur W',
 'E': 'Byculla',
 'T': 'Mulund',
 'KW': 'Andheri W',
 'FN': 'Matunga',
 'PN': 'Malad',
 'FS': 'Parel',
 'GN': 'Dadar',
 'HE': 'Khar',
 'S': 'Bhandup',
 'L': 'Kurla',
 'GS': 'Elphinstone',
 'KE': 'Andheri E',
 'N': 'Ghatkopar',
 'B': 'Sandhurst Road',
 'ME': 'Chembur E'}

In [28]:
len(ward_identifiers) == len(wards) # check that dictionary has all wards

True

pickle dictionary?

https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict

In [30]:
new_cases.drop(labels=0, inplace=True)

new_cases

,Date of report,RC,RN,RS,HW,A,PS,D,C,MW,...,GN,HE,S,L,GS,KE,N,B,ME,Grand Total
1,As on 16th Aug,6675,3272,6043,3519,3133,4248,5348,1798,3812,...,7582,4439,7066,5597,5620,8042,6657,1150,4520,125079.0
2,17-Aug,57,10,48,40,7,17,34,12,21,...,50,19,30,29,34,28,39,6,17,703.0
3,18-Aug,83,36,68,30,28,37,56,7,25,...,28,18,42,28,33,36,37,4,11,860.0
4,19-Aug,79,53,58,38,30,34,63,18,31,...,76,31,41,32,33,52,49,5,31,1050.0
5,20-Aug,161,48,84,39,27,43,65,13,42,...,71,35,38,45,37,65,39,3,22,1200.0
6,21-Aug,147,54,102,54,45,77,56,27,40,...,47,45,57,35,24,73,33,10,32,1324.0
7,22-Aug,129,51,70,32,52,51,49,17,46,...,41,20,57,37,42,45,31,12,24,1085.0
8,23-Aug,77,24,59,50,43,50,45,25,24,...,31,29,40,28,30,29,37,5,20,919.0
9,As on 23rd Aug,7408,3548,6532,3802,3365,4557,5716,1917,4041,...,7926,4636,7371,5831,5853,8370,6922,1195,4677,132220.0
10,Days to double,47,60,62,63,68,69,73,76,83,...,109,112,115,118,119,121,124,126,142,87.0


### Proactive screening of elderly to protect them and contain mortality

In [71]:
elderly_screening = tabula.read_pdf(file, pages=21, multiple_tables=False)

In [72]:
elderly = elderly_screening[0]

In [74]:
elderly.loc[2, 'Wards'] = 'Daily Totals'

In [75]:
elderly.drop(labels=[0,1], inplace=True)
elderly.drop(columns=['Unnamed: 6','Unnamed: 7'], inplace=True)

In [76]:
elderly.columns = [
    'Wards', 'Total No. of Houses Surveyed', 'Population Covered', 'Total No. of Senior Citizens', 'Sr Citizen SPO2>95','Sr Citizen SPO2<95'
]

In [84]:
elderly.loc[:, 'Sr Citizen SPO2<95'] = 0
elderly.loc[2:3, 'Sr Citizen SPO2<95'] = 1

In [88]:
elderly.index = elderly['Wards']
elderly.drop(columns=['Wards'], inplace=True)

In [90]:
elderly

,Total No. of Houses Surveyed,Population Covered,Total No. of Senior Citizens,Sr Citizen SPO2>95,Sr Citizen SPO2<95
Wards,,,,,
Daily Totals,"24,752","94,590","3,894","3,893",1
H/W,208,724,109,108,1
A,370,"1,458",11,11,0
B,370,"1,432",55,55,0
C,905,"3,714",54,54,0
D,426,"1,758",129,129,0
E,917,"3,198",152,152,0
F/S,0,0,0,0,0
F/N,0,0,0,0,0


**final step: save scraped data**